In [1]:
from alphablokus.files import *

In [19]:
file_list = list_files("s3://alpha-blokus/full/games/", ".bin")
print(file_list[-1])

file_list = list_files("s3://alpha-blokus/full/models/", ".onnx")
print(file_list[-1])

s3://alpha-blokus/full/games/2025-11-19_16-36-09-993259725-58185706_10007.bin
s3://alpha-blokus/full/models/04413189.onnx


In [12]:
file_list = list_files("s3://alpha-blokus/full/games/", ".bin")

files_to_keep = []
files_to_ignore = []

total_num_games = 0
for file in file_list:
    num_games = parse_num_games_from_filename(file)
    total_num_games += num_games

    if total_num_games <= 4413189:
        files_to_keep.append(file)
    else:
        print(file)
        files_to_ignore.append(file)

In [7]:
print(len(files_to_keep))
print(len(files_to_ignore))

441
22


In [8]:
s3 = boto3.client(
    "s3",
    aws_access_key_id=os.getenv("AWS_ACCESS_KEY_ID"),
    aws_secret_access_key=os.getenv("AWS_SECRET_ACCESS_KEY"),
    region_name=os.getenv("AWS_REGION_NAME"),
    endpoint_url=os.getenv("AWS_ENDPOINT_URL"),
)

In [4]:
localize_file(latest_file("s3://alpha-blokus/scrap/games/", ".bin"), "/tmp/shivam-blah")

'/tmp/shivam-blah/2025-09-28_18-38-58-010676000-58604857_59.bin'

In [9]:
import boto3
from botocore.exceptions import ClientError


s3 = boto3.client(
    "s3",
    aws_access_key_id=os.getenv("AWS_ACCESS_KEY_ID"),
    aws_secret_access_key=os.getenv("AWS_SECRET_ACCESS_KEY"),
    region_name=os.getenv("AWS_REGION_NAME"),
    endpoint_url=os.getenv("AWS_ENDPOINT_URL"),
)

def parse_s3_uri(uri: str) -> tuple[str, str]:
    """Return (bucket, key) from s3://bucket/key"""
    p = urlparse(uri)
    if p.scheme != "s3" or not p.netloc or not p.path:
        raise ValueError(f"Invalid S3 URI: {uri}")
    bucket = p.netloc
    key = p.path.lstrip("/")
    return bucket, key

def move_s3_object(s3_client, src_uri: str, dst_prefix_uri: str) -> str:
    """
    Move src_uri to dst_prefix_uri, preserving the filename.
    Returns the destination S3 URI.
    """
    src_bucket, src_key = parse_s3_uri(src_uri)

    dst_bucket, dst_prefix = parse_s3_uri(dst_prefix_uri)
    if not dst_prefix.endswith("/"):
        dst_prefix += "/"

    filename = src_key.rsplit("/", 1)[-1]
    dst_key = dst_prefix + filename

    copy_source = {"Bucket": src_bucket, "Key": src_key}

    # Copy (can be cross-bucket if desired)
    s3_client.copy_object(Bucket=dst_bucket, Key=dst_key, CopySource=copy_source)

    # Delete original (this is what makes it a "move")
    s3_client.delete_object(Bucket=src_bucket, Key=src_key)

    return f"s3://{dst_bucket}/{dst_key}"

def move_many(s3_client, s3_uris: list[str], dst_prefix_uri: str) -> dict[str, str]:
    """
    Moves all objects. Returns a mapping of src_uri -> dst_uri for successes.
    Raises at the end if any failures occurred.
    """
    successes: dict[str, str] = {}
    failures: list[tuple[str, str]] = []

    for src in s3_uris:
        try:
            dst = move_s3_object(s3_client, src, dst_prefix_uri)
            successes[src] = dst
            print(f"MOVED  {src}  ->  {dst}")
        except (ClientError, ValueError) as e:
            failures.append((src, str(e)))
            print(f"FAILED {src}: {e}")

    if failures:
        msg = "Some moves failed:\n" + "\n".join(f"- {src}: {err}" for src, err in failures)
        raise RuntimeError(msg)

    return successes

In [28]:
import os
from urllib.parse import urlparse

move_many(s3, paths, "s3://alpha-blokus/full/training_archive_1/")

MOVED  s3://alpha-blokus/full/training/04433194.pth  ->  s3://alpha-blokus/full/training_archive_1/04433194.pth
MOVED  s3://alpha-blokus/full/training/04453206.pth  ->  s3://alpha-blokus/full/training_archive_1/04453206.pth
MOVED  s3://alpha-blokus/full/training/04473225.pth  ->  s3://alpha-blokus/full/training_archive_1/04473225.pth
MOVED  s3://alpha-blokus/full/training/04493251.pth  ->  s3://alpha-blokus/full/training_archive_1/04493251.pth
MOVED  s3://alpha-blokus/full/training/04513266.pth  ->  s3://alpha-blokus/full/training_archive_1/04513266.pth
MOVED  s3://alpha-blokus/full/training/04533276.pth  ->  s3://alpha-blokus/full/training_archive_1/04533276.pth
MOVED  s3://alpha-blokus/full/training/04553304.pth  ->  s3://alpha-blokus/full/training_archive_1/04553304.pth
MOVED  s3://alpha-blokus/full/training/04573317.pth  ->  s3://alpha-blokus/full/training_archive_1/04573317.pth
MOVED  s3://alpha-blokus/full/training/04593332.pth  ->  s3://alpha-blokus/full/training_archive_1/04593

{'s3://alpha-blokus/full/training/04433194.pth': 's3://alpha-blokus/full/training_archive_1/04433194.pth',
 's3://alpha-blokus/full/training/04453206.pth': 's3://alpha-blokus/full/training_archive_1/04453206.pth',
 's3://alpha-blokus/full/training/04473225.pth': 's3://alpha-blokus/full/training_archive_1/04473225.pth',
 's3://alpha-blokus/full/training/04493251.pth': 's3://alpha-blokus/full/training_archive_1/04493251.pth',
 's3://alpha-blokus/full/training/04513266.pth': 's3://alpha-blokus/full/training_archive_1/04513266.pth',
 's3://alpha-blokus/full/training/04533276.pth': 's3://alpha-blokus/full/training_archive_1/04533276.pth',
 's3://alpha-blokus/full/training/04553304.pth': 's3://alpha-blokus/full/training_archive_1/04553304.pth',
 's3://alpha-blokus/full/training/04573317.pth': 's3://alpha-blokus/full/training_archive_1/04573317.pth',
 's3://alpha-blokus/full/training/04593332.pth': 's3://alpha-blokus/full/training_archive_1/04593332.pth',
 's3://alpha-blokus/full/training/046

In [25]:
model_list = list_files("s3://alpha-blokus/full/training/", ".pth")
print(len(model_list))


228


In [26]:
print(model_list[0])

s3://alpha-blokus/full/training/00000000.pth


In [27]:
paths = []
for model in model_list:
    if model > "s3://alpha-blokus/full/training/04413189.pth":
        print(model)
        paths.append(model)

s3://alpha-blokus/full/training/04433194.pth
s3://alpha-blokus/full/training/04453206.pth
s3://alpha-blokus/full/training/04473225.pth
s3://alpha-blokus/full/training/04493251.pth
s3://alpha-blokus/full/training/04513266.pth
s3://alpha-blokus/full/training/04533276.pth
s3://alpha-blokus/full/training/04553304.pth
s3://alpha-blokus/full/training/04573317.pth
s3://alpha-blokus/full/training/04593332.pth
s3://alpha-blokus/full/training/04613343.pth
